In [1]:
# source/s: 
# setup gspread
# https://www.youtube.com/watch?v=bu5wXjz2KvU
#
# setup pydrive
# https://www.youtube.com/watch?v=Z2kfNx3Cgsk&t=121s
#
# https://console.cloud.google.com

# pip install pydrive
# pip instll gspread
# pip install pandas
# pip install pydrive

import os
import pandas as pd
import numpy as np
import gspread
from datetime import datetime
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [2]:
# after sharing sheets with the service account robot email

# setup service account
sa = gspread.service_account(filename="sheets_credentials.json")

In [25]:
# # get the file name as datetime
# now = datetime.now()

# # https://stackoverflow.com/questions/10607688/how-to-create-a-file-name-with-the-current-date-time-in-python
# date_time_string = now.strftime("%Y%m%d-%H%M%S")

# print(date_time_string)

# # https://www.programiz.com/python-programming/datetime/strptime
# date_time_object = datetime.strptime(date_time_string, "%Y%m%d-%H%M%S")

20220505-195001


datetime.datetime

In [4]:
# # test to upload file
# filename = 'test.txt'
# filepath = backup_dir + filename
# gfile = drive.CreateFile({'parents' : [{'id' : folder}], 'title' : filename})
# # set content file is locking up test.txt
# # https://stackoverflow.com/questions/60132814/pydrive-upload-and-remove
# gfile.SetContentFile(filepath)
# gfile.Upload()
# gfile.content.close()

# # test to delete local file
# os.remove(filepath)
#

In [6]:
# open required sheets
#
# @ minimum column/s: {"YEAR", "RETAILER", "PROPERTY_TYPE", "TOTAL_DWELLING", "POSTCODE",
# "0L_DAY", "RATIO_0", "50L_DAY", "RATIO_50"}
sv_total_data_sh = sa.open("sv_total_data")
# @ minimum column/s: {"POSTCODE"}
holiday_postcodes_sh = sa.open("holiday_postcodes")
# @ minimum column/s: {"POSTCODE"}
exempt_postcodes_sh = sa.open("exempt_postcodes")
# @ minimum column/s = {"LOCALITY_NAME", "POSTCODE", "STATE"}
locality_postcodes_sh = sa.open("locality_postcode_index")


# open sheet worksheets
sv_total_data_ws = sv_total_data_sh.get_worksheet(0)
holiday_postcodes_ws = holiday_postcodes_sh.get_worksheet(0)
exempt_postcodes_ws = exempt_postcodes_sh.get_worksheet(0)
locality_postcodes_ws = locality_postcodes_sh.get_worksheet(0)

# get data from worksheets
sv_total_data_wsd = sv_total_data_ws.get_all_records()
holiday_postcodes_wsd = holiday_postcodes_ws.get_all_records()
exempt_postcodes_wsd = exempt_postcodes_ws.get_all_records()
locality_postcodes_wsd = locality_postcodes_ws.get_all_records()

# convert worksheet to pd.DataFrame
sv_total_data_df = pd.DataFrame.from_dict(sv_total_data_wsd)
holiday_postcodes_df = pd.DataFrame.from_dict(holiday_postcodes_wsd)
exempt_postcodes_df = pd.DataFrame.from_dict(exempt_postcodes_wsd)
locality_postcodes_df = pd.DataFrame.from_dict(locality_postcodes_wsd)

In [39]:
# setup pydrive
gauth = GoogleAuth()
drive = GoogleDrive(gauth)

# get the file name as datetime
now = datetime.now()
datetime_string = now.strftime("%Y%m%d-%H%M%S") + ".csv"

# local backup location
backup_dir = os.getcwd() + "\\backup\\"

# create csv file with sv total
sv_total_data_df.to_csv(backup_dir + datetime_string, index=False)

In [40]:
# test to upload file
# backup folder location IcZ6g4qCrSrd4miTeI"
folder = "1XawLc3-tpec5EdxcZ6g4qCrSrd4miTeI"
filepath = backup_dir + datetime_string

gfile = drive.CreateFile({'parents' : [{'id' : folder}], 'title' : datetime_string})

# set content file is locking up test.txt
# https://stackoverflow.com/questions/60132814/pydrive-upload-and-remove
gfile.SetContentFile(filepath)
gfile.Upload()

# update permissions of uploaded backup
permission = gfile.InsertPermission({'type':'user', 'role': 'reader', 'value':'cuestacodes@spec-vac.iam.gserviceaccount.com'})

# delete local file
gfile.content.close()
os.remove(filepath)

In [ ]:
# get datetime from string
date_time_object = datetime.strptime(datetime_string, "%Y%m%d-%H%M%S")

# TODO: find the latest datetime

In [7]:
sv_total_data_df.head()

,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,FALSE
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,FALSE
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,FALSE
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,FALSE
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,FALSE


In [8]:
holiday_postcodes_df.head()

,POSTCODE
0,3795
1,3797
2,3775
3,3777
4,3756


In [9]:
exempt_postcodes_df.head()

,POSTCODE
0,3020
1,3021
2,3022
3,3023
4,3024


In [10]:
locality_postcodes_df.head()

,LOCALITY_ID,LOCALITY_NAME,LOCALITY_TYPE,POSTCODE,STATE,SA2_MAINCODE,SA2_NAME
0,ACT107,ACTON,GAZETTED LOCALITY,2601,ACT,801000000.0,Acton
1,ACT107,ANU,ALIAS LOCALITY,2601,ACT,801000000.0,Acton
2,ACT107,AUSTRALIAN NATIONAL UNIVERSITY,ALIAS LOCALITY,2601,ACT,801000000.0,Acton
3,ACT107,SPINNAKER ISLAND,ALIAS LOCALITY,2601,ACT,801000000.0,Acton
4,ACT107,SPRINGBANK ISLAND,ALIAS LOCALITY,2601,ACT,801000000.0,Acton


In [11]:
# put holiday/exempt postcodes into sets
exempt_set = set(exempt_postcodes_df["POSTCODE"])
holiday_set = set(holiday_postcodes_df["POSTCODE"])

In [12]:
# re-do "EXEMPT_TAX" and "HOLIDAY" columns in total data
sv_total_data_df["EXEMPT_TAX_AREA"] = np.where(sv_total_data_df["POSTCODE"].isin(exempt_set), "exemption", "tax")
sv_total_data_df["HOLIDAY"] = np.where(sv_total_data_df["POSTCODE"].isin(holiday_set), True, False)

sv_total_data_df.head()

,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,False
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,False
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,False
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,False
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,False


In [13]:
# prepare locality names

# reduce locality df
locality_postcodes_df = locality_postcodes_df[["POSTCODE","LOCALITY_NAME", "STATE"]]

# filter for VIC postcodes
is_vic = locality_postcodes_df["STATE"] == "VIC"
locality_postcodes_df = locality_postcodes_df[is_vic]

In [14]:
# get count for each postcode
locality_postcodes_df["INC"] = locality_postcodes_df.groupby("POSTCODE").cumcount()
locality_postcodes_df.head(n=10)

,POSTCODE,LOCALITY_NAME,STATE,INC
25040,3737,ABBEYARD,VIC,0
25041,3067,ABBOTSFORD,VIC,0
25042,3067,COLLINGWOOD,VIC,1
25043,3067,FITZROY,VIC,2
25044,3040,ABERFELDIE,VIC,0
25045,3825,ABERFELDY,VIC,0
25046,3714,ACHERON,VIC,0
25047,3833,ADA,VIC,0
25048,3984,ADAMS ESTATE,VIC,0
25049,3352,ADDINGTON,VIC,0


In [15]:
# pivot using postcodes
locality_postcodes_df = locality_postcodes_df.pivot(index = ["POSTCODE"], values = "LOCALITY_NAME", columns = "INC")
locality_postcodes_df.head()

INC,0,1,2,3,4,5,6,7,8,9,...,59,60,61,62,63,64,65,66,67,68
POSTCODE,,,,,,,,,,,,,,,,,,,,,
0,MANNS BEACH,YARRAM,PORT ALBERT,OLD SETTLEMENT BEACH,PICNIC BEACH,PT ALBERT,SNAKE ISLAND,SNAKE IS,SWAN BAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2640,GATEWAY IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2648,YELTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2717,YELTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3002,EAST MELBOURNE,E MELBOURNE,JOLIMONT,MELBOURNE,MELBOURNE EAST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# create a column with the localities 
locality_postcodes_df['LOCALITY_CONCAT'] = locality_postcodes_df.apply(lambda row: row.dropna().tolist(), axis=1)
locality_postcodes_df.head()

INC,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,LOCALITY_CONCAT
POSTCODE,,,,,,,,,,,,,,,,,,,,,
0,MANNS BEACH,YARRAM,PORT ALBERT,OLD SETTLEMENT BEACH,PICNIC BEACH,PT ALBERT,SNAKE ISLAND,SNAKE IS,SWAN BAY,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[MANNS BEACH, YARRAM, PORT ALBERT, OLD SETTLEM..."
2640,GATEWAY IS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[GATEWAY IS]
2648,YELTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[YELTA]
2717,YELTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[YELTA]
3002,EAST MELBOURNE,E MELBOURNE,JOLIMONT,MELBOURNE,MELBOURNE EAST,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[EAST MELBOURNE, E MELBOURNE, JOLIMONT, MELBOU..."


In [17]:
# reduce locality df
locality_postcodes_df = pd.DataFrame({'POSTCODE':locality_postcodes_df.index, 'LOCALITY':locality_postcodes_df["LOCALITY_CONCAT"].values})
locality_postcodes_df.head()

,POSTCODE,LOCALITY
0,0,"[MANNS BEACH, YARRAM, PORT ALBERT, OLD SETTLEM..."
1,2640,[GATEWAY IS]
2,2648,[YELTA]
3,2717,[YELTA]
4,3002,"[EAST MELBOURNE, E MELBOURNE, JOLIMONT, MELBOU..."


In [18]:
# clean locality
locality_postcodes_df["LOCALITY"] = locality_postcodes_df["LOCALITY"].apply(lambda x: str(x).lstrip("[").rstrip(", '']").replace(",", " |").replace("'", ""))
locality_postcodes_df.head()

,POSTCODE,LOCALITY
0,0,MANNS BEACH | YARRAM | PORT ALBERT | OLD SETTL...
1,2640,GATEWAY IS
2,2648,YELTA
3,2717,YELTA
4,3002,EAST MELBOURNE | E MELBOURNE | JOLIMONT | MELB...


In [19]:
# join total and locality df
sv_total_data_df.merge(locality_postcodes_df, on='POSTCODE', how='left')


,YEAR,RETAILER,PROPERTY_TYPE,TOTAL_DWELLING,POSTCODE,0L_DAY,RATIO_0,50L_DAY,RATIO_50,EXEMPT_TAX_AREA,HOLIDAY,LOCALITY
0,2020,GWW,residential,25825,3000,331,1.281704,3339,12.92933,tax,False,NaN
1,2020,GWW,residential,2296,3002,11,0.479094,250,10.8885,tax,False,EAST MELBOURNE | E MELBOURNE | JOLIMONT | MELB...
2,2020,GWW,residential,4061,3003,35,0.861857,340,8.372322,tax,False,WEST MELBOURNE | MELBOURNE | MELBOURNE WEST | ...
3,2020,GWW,residential,6506,3008,75,1.152782,711,10.92837,tax,False,DOCKLANDS | WORLD TRADE CENTRE
4,2020,GWW,residential,10221,3011,190,1.858918,454,4.441835,tax,False,FOOTSCRAY | FOOTSCRAY WEST | SEDDON WEST | WES...
...,...,...,...,...,...,...,...,...,...,...,...,...
489,2020,YVW,non-residential,165,3793,11,6.666667,22,13.33333,exemption,True,MONBULK
490,2020,YVW,non-residential,95,3795,14,14.73684,9,9.473684,exemption,True,SILVAN
491,2020,YVW,non-residential,194,3796,10,5.154639,53,27.31959,exemption,False,MOUNT EVELYN | MT EVELYN
492,2020,YVW,non-residential,120,3797,11,9.166667,23,19.16667,exemption,True,GILDEROY | GLADYSDALE | POWELLTOWN | THREE BRI...


In [20]:
# save the loaded total dataframe as a backup_date sheet

In [21]:
# create a function that can bring back the total sheet to the last backup